In [211]:
import pandas as pd
from sqlalchemy import create_engine

# **Extract**

In [212]:
df_participant = pd.read_csv('https://storage.googleapis.com/dqlab-dataset/dqthon-participants.csv')
df_participant.head()

,participant_id,first_name,last_name,birth_date,address,phone_number,country,institute,occupation,register_time
0,bd9b6f88-b84f-4c4d-90f8-b67fe2f1a29a,Citra,Nurdiyanti,05 Feb 1991,"Gg. Monginsidi No. 08\nMedan, Aceh 80734",(0151) 081 2706,Georgia,UD Prakasa Mandasari,Business Intelligence Engineer,1.617634e+09
1,7dfe3391-6f40-47b6-b4db-0c76ebaf5fc3,Aris,Setiawan,11 Jan 1993,"Gg. Rajawali Timur No. 7\nPrabumulih, MA 09434",+62 (036) 461 7027,Korea Utara,Universitas Diponegoro,Frontend Engineer,1.617634e+09
2,19582d7f-b824-4fe5-a517-d5bf573fc768,Cornelia,Handayani,31 Jul 1993,"Jalan Kebonjati No. 0\nAmbon, SS 57739",089 833 6695,Komoro,UD Hardiansyah Puspasari,Business Analyst,1.617634e+09
3,aeb6d817-25f3-4867-8a74-8d92e0a0f633,Soleh,Rajasa,04 Nov 1991,"Jl. Yos Sudarso No. 109\nLubuklinggau, SR 76156",+62 (418) 329-4756,Eritrea,Perum Tampubolon Yuliarti,DevOps Engineer,1.617634e+09
4,1fdabdd9-5444-4c97-87b2-fe8833ad0d27,Vivi,Astuti,22 Jan 2003,"Jalan Gardujati No. 53\nKediri, Sulawesi Tenga...",0812511835,Aljazair,PT Hardiansyah Rahimah,Data Analyst,1.617634e+09


# **Transform**

In [213]:
# postal_code memuat informasi mengenai kode pos yang diambil dari alamat peserta (kolom address)
df_participant['postal_code'] = df_participant['address'].str.extract(r'([0-9]{5})')

# city merupakan sekumpulan karakter yang terdapat setelah nomor jalan
df_participant['city'] = df_participant['address'].str.extract(r'(?<=\n)(\w.+)(?=,)')

# github_profile merupakan gabungan dari firstname dan lastname yang di lowercase
df_participant['github_profile'] = 'https://github.com/' + df_participant['first_name'].str.lower() + df_participant['last_name'].str.lower()

# cleaned_phone_number merupakan format nomor handphone yang benar
df_participant['cleaned_phone_number'] = df_participant['phone_number'].str.replace(r'^(\+62|62)', '0')
df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'[()-]', '')
df_participant['cleaned_phone_number'] = df_participant['cleaned_phone_number'].str.replace(r'(\s+)', '')

# team_name merupakan gabungan nilai dari kolom first_name, last_name, country dan institute
def func(col):
    abbrev_name = "%s%s"%(col['first_name'][0], col['last_name'][0]) 
    country = col['country']
    abbrev_institute = '%s'%(''.join(list(map(lambda word: word[0], col['institute'].split())))) 
    return "%s-%s-%s"%(abbrev_name,country,abbrev_institute)
df_participant['team_name'] = df_participant.apply(func, axis=1)

# email merupakan gabungan dari first name, last name dan singkatan institute
def func(col):
    first_name_lower = col['first_name'].lower()
    last_name_lower = col['last_name'].lower()
    institute = ''.join(list(map(lambda word: word[0], col['institute'].lower().split()))) 

    if 'Universitas' in col['institute']:
        if len(col['country'].split()) > 1: 
            country = ''.join(list(map(lambda word: word[0], col['country'].lower().split())))
        else:
            country = col['country'][:3].lower()
        return "%s%s@%s.ac.%s"%(first_name_lower, last_name_lower, institute,country)

    return "%s%s@%s.com"%(first_name_lower, last_name_lower, institute)
df_participant['email'] = df_participant.apply(func, axis=1)

# birth_date diubah sesuai format
df_participant['birth_date'] = pd.to_datetime(df_participant['birth_date'], format = '%d %b %Y')

# register_at 
df_participant['register_at'] = pd.to_datetime(df_participant['register_time'], unit='s')

df_participant.head()

,participant_id,first_name,last_name,birth_date,address,phone_number,country,institute,occupation,register_time,postal_code,city,github_profile,cleaned_phone_number,team_name,email,register_at
0,bd9b6f88-b84f-4c4d-90f8-b67fe2f1a29a,Citra,Nurdiyanti,1991-02-05,"Gg. Monginsidi No. 08\nMedan, Aceh 80734",(0151) 081 2706,Georgia,UD Prakasa Mandasari,Business Intelligence Engineer,1.617634e+09,80734,Medan,https://github.com/citranurdiyanti,01510812706,CN-Georgia-UPM,citranurdiyanti@upm.com,2021-04-05 14:47:26
1,7dfe3391-6f40-47b6-b4db-0c76ebaf5fc3,Aris,Setiawan,1993-01-11,"Gg. Rajawali Timur No. 7\nPrabumulih, MA 09434",+62 (036) 461 7027,Korea Utara,Universitas Diponegoro,Frontend Engineer,1.617634e+09,09434,Prabumulih,https://github.com/arissetiawan,00364617027,AS-Korea Utara-UD,arissetiawan@ud.ac.ku,2021-04-05 14:46:58
2,19582d7f-b824-4fe5-a517-d5bf573fc768,Cornelia,Handayani,1993-07-31,"Jalan Kebonjati No. 0\nAmbon, SS 57739",089 833 6695,Komoro,UD Hardiansyah Puspasari,Business Analyst,1.617634e+09,57739,Ambon,https://github.com/corneliahandayani,0898336695,CH-Komoro-UHP,corneliahandayani@uhp.com,2021-04-05 14:47:15
3,aeb6d817-25f3-4867-8a74-8d92e0a0f633,Soleh,Rajasa,1991-11-04,"Jl. Yos Sudarso No. 109\nLubuklinggau, SR 76156",+62 (418) 329-4756,Eritrea,Perum Tampubolon Yuliarti,DevOps Engineer,1.617634e+09,76156,Lubuklinggau,https://github.com/solehrajasa,04183294756,SR-Eritrea-PTY,solehrajasa@pty.com,2021-04-05 14:47:14
4,1fdabdd9-5444-4c97-87b2-fe8833ad0d27,Vivi,Astuti,2003-01-22,"Jalan Gardujati No. 53\nKediri, Sulawesi Tenga...",0812511835,Aljazair,PT Hardiansyah Rahimah,Data Analyst,1.617634e+09,70074,Kediri,https://github.com/viviastuti,0812511835,VA-Aljazair-PHR,viviastuti@phr.com,2021-04-05 14:46:50


# **Load**

In [214]:
engine = create_engine('sqlite://', echo=False)
df_participant.to_sql('dqthon', con=engine, if_exists='replace', index = False)
engine.execute("SELECT * FROM dqthon").fetchall()

[('bd9b6f88-b84f-4c4d-90f8-b67fe2f1a29a', 'Citra', 'Nurdiyanti', '1991-02-05 00:00:00.000000', 'Gg. Monginsidi No. 08\nMedan, Aceh 80734', '(0151) 081 2706', 'Georgia', 'UD Prakasa Mandasari', 'Business Intelligence Engineer', 1617634046.0, '80734', 'Medan', 'https://github.com/citranurdiyanti', '01510812706', 'CN-Georgia-UPM', 'citranurdiyanti@upm.com', '2021-04-05 14:47:26.000000'),
 ('7dfe3391-6f40-47b6-b4db-0c76ebaf5fc3', 'Aris', 'Setiawan', '1993-01-11 00:00:00.000000', 'Gg. Rajawali Timur No. 7\nPrabumulih, MA 09434', '+62 (036) 461 7027', 'Korea Utara', 'Universitas Diponegoro', 'Frontend Engineer', 1617634018.0, '09434', 'Prabumulih', 'https://github.com/arissetiawan', '00364617027', 'AS-Korea Utara-UD', 'arissetiawan@ud.ac.ku', '2021-04-05 14:46:58.000000'),
 ('19582d7f-b824-4fe5-a517-d5bf573fc768', 'Cornelia', 'Handayani', '1993-07-31 00:00:00.000000', 'Jalan Kebonjati No. 0\nAmbon, SS 57739', '089 833 6695', 'Komoro', 'UD Hardiansyah Puspasari', 'Business Analyst', 161763403